In [ ]:
import os
import sys
import pandas as pd
import requests, re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.stats import fisher_exact

In [ ]:
# Creating the list of gene names from ACMG list, drawn from NCBI website

acmg_url = 'https://www.ncbi.nlm.nih.gov/clinvar/docs/acmg/'

response = requests.get(acmg_url)
soup = BeautifulSoup(response.content, "html.parser")
results = soup.find(id="maincontent")

column_index = 2
acmg_genes = []

for cell in results.find_all('td'):
    if cell.find('a') and 'genes' in cell.find('a').get('href', ''):
        gene = cell.text.strip()
        # Cleaning the gene name by removing any content in parentheses
        gene = re.sub(r'\([^)]*\)', '', gene).strip()
        acmg_genes.append(gene)

acmg_set = set(acmg_genes)

In [ ]:
def fetch_endpoint(server, request, content_type):

    r = requests.get(server+request, headers={ "Accept" : content_type})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    if content_type == 'application/json':
        return r.json()
    else:
        return r.text

In [ ]:
# Function for getting the gene positions from Ensembl database

def get_gene(gene: str, grch: str='grch37'):

    # Getting the Ensembl server and extension
    if grch == 'grch37':
        ens_serv: str = f'http://{grch}.rest.ensembl.org/'
    elif grch == 'grch38':
        ens_serv: str = f'http://rest.ensembl.org/'
    else:
        raise Exception('Please provide a correct grch assembly! (grch37/grch38 are supported)')
    
    # Connecting to server and getting data
    ens_ext: str = f'lookup/symbol/homo_sapiens/{gene}'
    con: str = 'application/json'

    return fetch_endpoint(ens_serv, ens_ext, con)

In [ ]:
# Writing ACMG-listed gene positions into a new file

output_file = 'regions.bed'

os.system(f'touch {output_file}')
with open(f'{output_file}', 'a') as f:
    for idx, gene in reversed(list(enumerate(acmg_set))):
        gene_json = get_gene(gene)
        chromosome = gene_json['seq_region_name']
        start = gene_json['start'] - 10000  # Ensuring comprehensive coverage of variant positions of the genes
        end = gene_json['end'] + 10000   # Ensuring comprehensive coverage of variant positions of the genes
        location = f'{chromosome}\t{start}\t{end}'
        if idx > 0:
            f.write(location)
            f.write('\n')
        else:
            f.write(location)

In [ ]:
# Extracting ACMG-listed gene positions from the gnomAD database

os.system(f'bcftools view --regions-file regions.bed -Oz gnomad.genomes.r2.1.1.sites.vcf.bgz > gnomad.genomes.regions.r2.1.1.sites.vcf.gz')
os.system(f'bcftools index gnomad.genomes.regions.r2.1.1.sites.vcf.gz')

In [ ]:
# Changing the format of the reduced database - to exclude the extensive information

os.system(f'bcftools query -f "%CHROM\t%POS\t%ID\t%REF\t%ALT\t%QUAL\t%FILTER\t%INFO/AF;%INFO/vep\n" -o extracted_gnomad.genomes.regions.r2.1.1.sites.vcf.gz gnomad.genomes.regions.r2.1.1.sites.vcf.gz')


In [ ]:
# Fields for annotation

fields = "AF,MAX_AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,Existing_variation,Allele,Consequence,Gene,NMD,SYMBOL,SYMBOL_SOURCE,IMPACT,ClinVar,ClinVar_CLNREVSTAT,ClinVar_AF_EXAC,ClinVar_CLNDISDB,ClinVar_CLNDISDBINCL,ClinVar_CLNSIG,ClinVar_CLNSIGCONF,ClinVar_CLNSIGINCL,ClinVar_CLNVC,ClinVar_GENEINFO,ClinVar_RS,gnomADg,gnomADg_AN,gnomADg_AC,gnomADg_AF,PUBMED,SIFT,PolyPhen,BIOTYPE,Feature_type,Feature,CLIN_SIG"

In [ ]:
# VEP annotation

os.system(f' vep --offline -i extracted_gnomad.genomes.regions.r2.1.1.sites.vcf.gz --assembly GRCh37 --vcf --af --max_af --af_1kg \
--fields {fields} --per_gene -o out_extracted_gnomad.vcf --force_overwrite --fork 5 --plugin NMD \
--custom /data/clinvar.vcf.gz,ClinVar,vcf,exact,0,CLNREVSTAT,AF_EXAC,CLNDISDB,CLNDISDBINCL,CLNSIG,CLNSIGCONF,CLNSIGINCL,CLNVC,GENEINFO,RS \
--custom file=gnomad.genomes.regions.r2.1.1.sites.vcf.gz,short_name=gnomADg,format=vcf,type=exact,coords=0,fields=AN%AC%AF%AF_afr%AF_amr%AF_asj%AF_eas%AF_fin%AF_nfe%AF_oth')

In [ ]:
# Function for reading VCF files

def read_genotyped_file(path_to_file: str, separators=["\t", ","], possible_number_of_cols = [4,5],
                        possible_header_starts = ["RSID", "rsid", "#CHROM", "rsID"]) -> pd.DataFrame:

    with open(path_to_file, 'r') as f:
        header = None
        for line in f:
            if line.startswith('#'):
                header = line
            else:
                if header is None:
                    header = line
                break # Stop when there are no more

    is_header = any([(header_start in header) for header_start in possible_header_starts])

    if is_header:
        for separator in separators:
            header_names = header[1:]
            header_names = header_names.strip().split(separator)
            if len(header_names)==1:
                continue

            data = pd.read_csv(path_to_file, sep=separator, comment='#', names=header_names)

            if data.shape[1] in possible_number_of_cols:
                break

    else:
        for separator in separators:
            data = pd.read_csv(path_to_file, sep=separator, comment='#')

            if data.shape[1] in possible_number_of_cols:
                break

    return data

In [ ]:
# Filtering out variants with AF above 0.05

df1 = read_genotyped_file('out_extracted_gnomad.vcf')
df_5 = df1[
    (df1['INFO'].apply(lambda x: float(x.split(';')[0])) <= 0.05) |
    (df1['INFO'].apply(lambda x: float(x.split(';')[0])).isna())
]
df_5.to_csv('below5_extracted_gnomad.csv', index=False, header=False)

In [ ]:
# Reading the variant database from ClinVar
df_clinvar = pd.read_csv(f'clinvar/variant_summary.txt', sep='\t')

# Filtering out variants with GRCh38 assembly from the ClinVar database and creating a position ID (POS_ID) column
df_clinvar = df_clinvar[df_clinvar['Assembly'] != 'GRCh38']
df_clinvar['POS_ID'] = df_clinvar.apply(lambda row: f"{row['Chromosome']}-{row['PositionVCF']}-{row['ReferenceAlleleVCF']}-{row['AlternateAlleleVCF']}", axis=1)


In [ ]:
# Columns in the new format
column_names = ['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO']

In [ ]:
df_gnom = pd.read_csv(f'below5_extracted_gnomad.csv', sep=',', names=column_names)

# Creating a position ID (POS_ID) column
df_gnom['POS_ID'] = df_gnom.apply(lambda row: f"{row['CHROM']}-{row['POS']}-{row['REF']}-{row['ALT']}", axis=1)

# Merging gnomAD and ClinVar databases
merged_df = pd.merge(df_gnom, df_clinvar, on='POS_ID', how='left')
merged_df.to_csv(f'clinvar_extracted_gnomad.csv', index=False, header=False)

In [ ]:
# New columns names including the columns from ClinVar database
columns = ['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','POS_ID','AlleleID','Type','Name','GeneID','GeneSymbol','HGNC_ID','ClinicalSignificance','ClinSigSimple','LastEvaluated','RS# (dbSNP)','nsv/esv (dbVar)','RCVaccession','PhenotypeIDS','PhenotypeList','Origin','OriginSimple','Assembly','ChromosomeAccession','Chromosome','Start','Stop','ReferenceAllele','AlternateAllele','Cytogenetic','ReviewStatus','NumberSubmitters','Guidelines','TestedInGTR','OtherIDs','SubmitterCategories','VariationID','PositionVCF','ReferenceAlleleVCF','AlternateAlleleVCF']

In [ ]:
# Filtering variants with HIGH impact or Pathogenic/Likely pathogenic clinical significance

df_clin = pd.read_csv(f'clinvar_extracted_gnomad.csv', sep=',', names=columns)
final_clin_df = df_clin[df_clin['INFO'].str.contains('HIGH') | df_clin['ClinicalSignificance'].str.contains('Pathogenic') | df_clin['ClinicalSignificance'].str.contains('Pathogenic/Likely pathogenic') | df_clin['ClinicalSignificance'].str.contains('Likely pathogenic')]
final_clin_df.to_csv(f'cv_patho_high.csv', index=False, header=False)

In [ ]:
# Finding RS IDs of the filtered variants to find the matching rows in the gnomAD database

df = pd.read_csv('cv_patho_high.csv', sep=',', names=columns)
ids = df['ID']   # Finding all IDs
ids.to_csv('rs_ids_cv_patho_high.txt', index=False, header=False)

with open('rs_ids_cv_patho_high.txt', 'r') as file:
    lines = file.readlines()

filtered_lines = [line.strip() for line in lines if '.' not in line]     # Finding only known IDs - so not '.'

with open('rs_ids.txt', 'w') as file:
    for line in filtered_lines:
        file.write(line + '\n')

In [ ]:
df = pd.read_csv('cv_patho_high.csv', sep=',', names=columns)
filtered_df = df[df['ID'] == '.']    # Finding all rows with '.' as ID
filtered_values = filtered_df[['CHROM', 'POS']]     # Finding positions of variants in these rows and writing them into a file
filtered_values.to_csv('rs_ids_unknown.txt', sep='\t', index=False, header=False)

In [ ]:
# Finding the matching rows in the gnomAD database

os.system(f"bcftools view -i ID=@rs_ids.txt gnomad.genomes.regions.r2.1.1.sites.vcf.gz > gnomad_zip_rows_tomatch.vcf")   # By RS IDs
os.system(f"bcftools view -R 'rs_ids_unknown.txt' gnomad.genomes.regions.r2.1.1.sites.vcf.gz > gnomad_zip_rows_tomatch_unknown.vcf")  # By positions

In [ ]:
df_gnomad = read_genotyped_file('gnomad_zip_rows_tomatch.vcf')
df_gnomad_2 = read_genotyped_file('gnomad_zip_rows_tomatch_dots.vcf')

# Merging the rows from both files to later attach previously left out information from INFO column
combined_df = pd.concat([df_gnomad, df_gnomad_2], ignore_index=True)

# Merging the information from both dataframes
df_clin = pd.read_csv('cv_patho_high.csv', sep=',', names=columns)
combined_df['POS_ID'] = combined_df.apply(lambda row: f"{row['CHROM']}-{row['POS']}-{row['REF']}-{row['ALT']}", axis=1)
merged_df = pd.merge(df_clin, combined_df, on='POS_ID', how='left')
merged_df.to_excel('cv_patho_high_full_info.xlsx', index=False)

In [ ]:
# Extracting information from INFO column into separate columns for easy filtering

def extract_info(row, chosen_fields):
    csq_info = row['INFO_x'].split('CSQ=')[1]
    csq_fields = csq_info.split('|')
    chosen_fields = [csq_fields[i] if csq_fields[i] else '-' for i in chosen_fields]
    return pd.Series(chosen_fields)

def process_dataframe(df, chosen_indices, new_column_names):
    new_columns = df.apply(extract_info, args=(chosen_indices,), axis=1)
    new_columns.columns = new_column_names

    # Concatenating the new columns with the original df
    df = pd.concat([df.loc[:, :'INFO_x'], new_columns, df.loc[:, 'INFO_x':]], axis=1)
    df = df.loc[:, ~df.columns.duplicated()]

    return df

In [ ]:
df = pd.read_excel('cv_patho_high_full_info.xlsx')
df = df[df['Assembly'] != 'GRCh38']

# Choosing the names for new columns (names drawn from VEP) and their respective indices in the INFO column
chosen_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
new_column_names = ['AF','MAX_AF','AFR_AF','AMR_AF','EAS_AF','EUR_AF','SAS_AF','Existing_variation','Allele','Consequence','Gene','NMD','SYMBOL','SYMBOL_SOURCE','IMPACT','ClinVar','ClinVar_CLNREVSTAT','ClinVar_AF_EXAC','ClinVar_CLNDISDB','ClinVar_CLNDISDBINCL','ClinVar_CLNSIG','ClinVar_CLNSIGCONF','ClinVar_CLNSIGINCL','ClinVar_CLNVC','ClinVar_GENEINFO','ClinVar_RS','gnomADg','gnomADg_AN','gnomADg_AC','gnomADg_AF','PUBMED','SIFT','PolyPhen','BIOTYPE','Feature_type','Feature','CLIN_SIG']

# Processing the dataframe to extract the new columns
df_all = process_dataframe(df, chosen_indices, new_column_names)

# Filtering out rows with 'NMD_escaping_variant' in NMD column
filtered_df = df_all[df_all['NMD'] != 'NMD_escaping_variant']

filtered_df.to_excel('final_table.xlsx', index=False)

In [ ]:
df = pd.read_excel('final_table.xlsx')

# Finding the AC and AN values for a population

def extract_population_freq(info_string, population):
    pattern = f"AC_{population}=([0-9]+);AN_{population}=([0-9]+)"
    match = re.search(pattern, info_string)
    if match:
        ac, an = match.groups()
        return float(ac), float(an)
    else:
        return None, None
    
populations = ['amr', 'asj', 'afr', 'eas', 'fin', 'nfe_nwe', 'nfe_seu', 'nfe_onf', 'nfe_est'] 
for population in populations:
    ac_values = []
    an_values = []
    for index, row in df.iterrows():
        ac, an = extract_population_freq(row['INFO_y'], population) 
        ac_values.append(ac)
        an_values.append(an)
    # Adding new columns in the dataframe - for AC and AN values per population
    df[f'AC_{population}'] = ac_values
    df[f'AN_{population}'] = an_values
    
for population in populations:
    df[f'Frequency_{population}'] = df[f'AC_{population}'] / df[f'AN_{population}']   # Adding another column for the AC/AN ratio per population
df.to_excel('populations_final.xlsx', index=False)

### Statistical analysis

In [ ]:
df = pd.read_excel('populations_final.xlsx')

# Removing duplicate rows based on the POS_ID column - if they exist
df_cleaned = df.drop_duplicates(subset=['POS_ID'])

df_cleaned.to_excel('cleaned_file_posid.xlsx', index=False)

In [ ]:
df = pd.read_excel('cleaned_file_posid.xlsx')

# Filtering rows where number of submitters is 2 or more, or is NaN
df_filtered = df[(df['NumberSubmitters'] >= 2) | (df['NumberSubmitters'].isna())]

# Filtering out variants with conflicting interpretations of pathogenicity
df_filtered_conflics = df_filtered[df_filtered['ClinicalSignificance'] != 'Conflicting interpretations of pathogenicity']

df_filtered_conflics.to_excel('filtered_more_submits.xlsx', index=False)

In [ ]:
# CV_P+HIGH_VEP experiment

df = pd.read_excel('filtered_more_submits.xlsx')

lst = ['Pathogenic','Pathogenic/Likely pathogenic','Likely pathogenic','Pathogenic; drug response','Likely pathogenic; drug response']

# Filtering rows where clinical significance is included in the lst list or is NaN
df_filtered_ag = df_filtered.loc[(df_filtered['ClinicalSignificance'].isin(lst)) | (df_filtered['ClinicalSignificance'].isna()),]

df_filtered_ag.to_excel('cv_p_high_vep.xlsx' , index=False)

In [ ]:
# CV_P experiment

df = pd.read_excel('filtered_more_submits.xlsx')

lst = ['Pathogenic','Pathogenic/Likely pathogenic','Likely pathogenic','Pathogenic; drug response','Likely pathogenic; drug response']

# Filtering rows where clinical significance is included in the lst list
df_filtered_ag = df_filtered.loc[(df_filtered['ClinicalSignificance'].isin(lst)),]

df_filtered_ag.to_excel('cv_p.xlsx' , index=False)

In [ ]:
# Manually choosing one or the other experiment

df = pd.read_excel('cv_p_high_vep.xlsx')  # CV_P+HIGH_VEP experiment
# df = pd.read_excel('cv_p.xlsx')  # CV_P experiment

In [ ]:
an_columns = ['AN_amr', 'AN_asj', 'AN_afr', 'AN_eas', 'AN_fin', 'AN_nfe_nwe', 'AN_nfe_seu', 'AN_nfe_est', 'AN_nfe_onf'] 
ac_columns = ['AC_amr', 'AC_asj', 'AC_afr', 'AC_eas', 'AC_fin', 'AC_nfe_nwe', 'AC_nfe_seu', 'AC_nfe_est', 'AC_nfe_onf']  

# Initializing dictionaries to store median and sum results
median_results_an = {}
sums_ac = {}
ratios_median = {}

for column in an_columns:
    # Calculating the median of the current AN column within each group
    median_results_an[column] = df.groupby('SYMBOL')[column].median().reset_index()
    
for column in ac_columns:
    # Calculating the sum of the current AC column within each group
    sums_ac[column] = df.groupby('SYMBOL')[column].sum().reset_index()

results_median = []

for key in sums_ac.keys():
    ac_df = sums_ac[key]
    an_df = median_results_an[key.replace("AC", "AN")]
    
    # Merging the DataFrames on 'SYMBOL' column
    merged_df = pd.merge(ac_df, an_df, on='SYMBOL')
    
    # Dividing AC sums by AN medians
    merged_df[f'AC/AN_ratio_{key}'] = merged_df[key] / merged_df[key.replace("AC", "AN")]
    
    results_median.append(merged_df)

In [ ]:
# Creating dictionary with population names as keys and their respective dataframes as items

df_dict_median = {}

for df in results_median:
    for column in df.columns:
        if column.startswith('AN_'):
            column = column.lstrip('AN_')
            df_dict_median[column] = df
            break

In [ ]:
# Full populations names
full_names_pop = {'afr':'African','amr':'Latino','asj':'Ashkenazi Jews', 'eas':'East Asian','fin':'Finnish','nfe_nwe':'North-Western European','nfe_seu':'Southern European','nfe_onf':'Other Non-Finnish European','nfe_est':'Estonian'}

In [ ]:
# Preparing data for Fisher's Exact Test

population_info = {}

for population, df in df_dict_median.items():
    filtered_df = df[df['SYMBOL'].isin(acmg_genes)]

    population_df = filtered_df[['SYMBOL', f'AC_{population}', f'AN_{population}']]

    # Calculating difference between the median of AN values per gene the sum of AC values
    population_df[f'AN-AC_{population}'] = filtered_df[f'AN_{population}'] - filtered_df[f'AC_{population}']
    
    # Converting the dataframe to a list of dictionaries
    population_list = population_df.to_dict(orient='records')
    
    population_info[population] = population_list

In [ ]:
# Performing Fisher's Exact Test for each gene

fisher_results = {}

# Iterating through each gene
for symbol in acmg_genes:
    if symbol not in [data_dict['SYMBOL'] for data_list in population_info.values() for data_dict in data_list]:
        continue  # Skip the symbol if it has no variants

    symbol_results = {}
    
    # Generating all pairs of populations
    population_pairs = combinations(population_info.keys(), 2)
    
    # Iterating through each pair of populations
    for population_a, population_b in population_pairs:
        data_a = population_info[population_a]
        data_b = population_info[population_b]
        
        # Initializing contingency table
        contingency_table = [[], []]
        
        # Filling contingency table with AC and AN-AC values for the current symbol
        for data_dict_a, data_dict_b in zip(data_a, data_b):
            if data_dict_a['SYMBOL'] == symbol and data_dict_b['SYMBOL'] == symbol:
                contingency_table[0].append(data_dict_a[f'AC_{population_a}'])
                contingency_table[1].append(data_dict_b[f'AC_{population_b}'])
                contingency_table[0].append(data_dict_a[f'AN-AC_{population_a}'])
                contingency_table[1].append(data_dict_b[f'AN-AC_{population_b}'])
        
        # Performing Fisher's exact test
        odds_ratio, p_value = fisher_exact(contingency_table)
        symbol_results[(population_a, population_b)] = {'odds_ratio': odds_ratio, 'p_value': p_value}
    
    # Storing Fisher's exact test results for the current symbol
    fisher_results[symbol] = symbol_results

In [ ]:
# Creating dictionary with phenotypes as keys and lists of corresponding genes as items

groups_dict = {}

with open('genes phenotypes groups.txt', 'r') as file:    # The .txt file based on the table from Miller et al. (2023)
    current_group = None
    for line in file:
        line = line.strip()
        if not line:
            # If the line is empty, it indicates a new group
            current_group = None
        else:
            # If the line is not empty
            if current_group is None:
                # If current_group is None, it indicates a new group
                current_group = line
                # Initializing a list for the current group
                groups_dict[current_group] = []
            else:
                # If current_group is not None, appending the symbol to the list of the current group
                groups_dict[current_group].append(line)

In [ ]:
# Preparing the dictionary with genes grouped by phenotype for Fisher's Exact Test

results_grouped_dict = {}

for population, df in df_dict_median.items():
    ac_sums = {}
    ac_minus_an_median = {}
    
    for group, symbols in groups_dict.items():
        # Filtering the dataframe to include only symbols in the current group
        group_df = df[df['SYMBOL'].isin(symbols)]
        
        # Calculating the sum of AC values for the current group
        ac_sum = group_df[f'AC_{population}'].sum()
        ac_sums[group] = ac_sum
        
        # Calculating difference between the median of AN values per gene the sum of AC values
        ac_minus_an_median[group] = group_df[f'AN_{population}'].median() - group_df[f'AC_{population}'].sum()
    
    results_grouped_dict[population] = {'AC_sums': ac_sums, f'AN-AC_{population}': ac_minus_an_median}

In [ ]:
# Performing Fisher's Exact Test for each phenotype (group)

fisher_results_grouped = {}

for group_name in list(results_grouped_dict[next(iter(results_grouped_dict))]['AC_sums'].keys()):
    fisher_results_grouped[group_name] = {}
    
    # Iterating through each pair of populations
    for population_pair in combinations(results_grouped_dict.keys(), 2):
        population_a, population_b = population_pair
        
        # Population A
        ac_sum_a = results_grouped_dict[population_a]['AC_sums'][group_name]
        an_minus_ac_a = results_grouped_dict[population_a]['AN-AC_' + population_a][group_name]
        
        # Population B
        ac_sum_b = results_grouped_dict[population_b]['AC_sums'][group_name]
        an_minus_ac_b = results_grouped_dict[population_b]['AN-AC_' + population_b][group_name]
        
        # Constructing the contingency table
        contingency_table = [
            [ac_sum_a, an_minus_ac_a],
            [ac_sum_b, an_minus_ac_b]
        ]
        
        # Performing Fisher's Exact Test
        odds_ratio, p_value = fisher_exact(contingency_table)
        
        fisher_results_grouped[group_name][population_pair] = {'odds_ratio': odds_ratio, 'p_value': p_value}

In [ ]:
# Creating plots for each gene

for symbol in acmg_genes:
    if symbol in fisher_results.keys():
        total_ac_an_ratio = 0
        for column, median_df in df_dict_median.items():
            pop = column
            symbol_median_df = median_df[median_df['SYMBOL'] == symbol]
            if not symbol_median_df.empty:
                total_ac_an_ratio += symbol_median_df[f'AC/AN_ratio_AC_{pop}'].iloc[0]

        # Skip plotting if total AC/AN ratio is zero - no variants of the given gene
        if total_ac_an_ratio == 0:
            continue

        plt.figure(figsize=(15, 17))
        plt.title(f'Frequency of individuals with a variant present in {symbol} gene', fontsize=20)
        plt.xlabel('Populations', fontsize=16)
        plt.ylabel('Frequency', fontsize=16)

        for i, (column, median_df) in enumerate(df_dict_median.items()):
            pop = column
            symbol_median_df = median_df[median_df['SYMBOL'] == symbol]
            plt.bar(i, symbol_median_df['AC/AN_ratio_AC_' + pop], alpha=0.8, label=f'{column}')

        # Adding horizontal lines representing p-values between different population pairs
        for i, ((pop1, pop2), p_value_data) in enumerate(fisher_results[symbol].items()):
            if p_value_data['p_value'] < 0.05:
                p_value = p_value_data['p_value']
                x1 = list(df_dict_median.keys()).index(pop1)
                x2 = list(df_dict_median.keys()).index(pop2)
                y = max(plt.ylim())
                plt.plot([x1, x2], [y, y], linewidth=2, color='black')  # Horizontal line
                p_value_label = f'{p_value:.4f}' if p_value >= 1e-4 else f'<{0.0001}'  # Writing p-value on the line if it's between 0.05 and 0.0001, if it's smaller, writing '<0.0001'
                plt.text((x1 + x2) / 2, y, f'p-value: {p_value_label}', ha='center', va='bottom', fontsize=12)

        plt.xticks(range(len(df_dict_median)), [full_names_pop[pop] for pop in df_dict_median.keys()], rotation=-45, fontsize=12)

        plt.tight_layout() 

        # Choosing the directory depending on the experiment
        plt.savefig(f'plots_cv_p_high_vep/genes/{symbol}.jpg')   # CV_P+HIGH_VEP experiment
        # plt.savefig(f'plots_cv_p/genes/{symbol}.jpg')    # CV_P experiment
        plt.close()

In [ ]:
# Calculating AC/AN ratios per phenotypes (groups)

disease_ratios = {}

for disease, symbols in groups_dict.items():
    disease_df = pd.DataFrame()
    for symbol in symbols:
        symbol_data = pd.concat([data[data['SYMBOL'] == symbol] for _, data in df_dict_median.items()])
        disease_df = pd.concat([disease_df, symbol_data])
    
    # Calculating sum of AC values and median of AN values for each population
    disease_ratios[disease] = {}
    for population, data in df_dict_median.items():
        population_subset = disease_df[disease_df['SYMBOL'].isin(symbols)]
        ac_sum = population_subset[f'AC_{population}'].sum()
        an_median = population_subset[f'AN_{population}'].median()
        # Calculating new AC/AN ratio
        if an_median != 0:
            ratio = ac_sum / an_median
        else:
            ratio = None  # Handling case where denominator is 0
        disease_ratios[disease][population] = ratio

In [ ]:
# Creating plots for each phenotype

for disease, ratios_per_population in disease_ratios.items():
    plt.figure(figsize=(15, 20))
    plt.title(f'Frequency of individuals with a variant present in genes\nrelated to {disease}', fontsize=20, pad=20)
    plt.xlabel('Populations', fontsize=16)
    plt.ylabel('Frequency', fontsize=16)

    for i, (population, ratio) in enumerate(ratios_per_population.items()):
        plt.bar(i, ratio, alpha=0.8, label=f'{population}')
    
    # Adding horizontal lines representing p-values between different population pairs
    if disease in fisher_results_grouped:
        for i, ((pop1, pop2), p_value_data) in enumerate(fisher_results_grouped[disease].items()):
            if p_value_data['p_value'] < 0.05:
                p_value = p_value_data['p_value']
                x1 = list(ratios_per_population.keys()).index(pop1)
                x2 = list(ratios_per_population.keys()).index(pop2)
                y = max(plt.ylim())
                plt.plot([x1, x2], [y, y], linewidth=2, color='black')  # Horizontal line
                p_value_label = f'{p_value:.4f}' if p_value >= 1e-4 else f'<{0.0001}'  # Writing p-value on the line if it's between 0.05 and 0.0001, if it's smaller, writing '<0.0001'
                plt.text((x1 + x2) / 2, y, f'p-value: {p_value_label}', ha='center', va='bottom', fontsize=12)

    plt.xticks(range(len(ratios_per_population)), [full_names_pop[pop] for pop in ratios_per_population.keys()], rotation=-45, fontsize=12)

    plt.tight_layout() 

    # Choosing the directory depending on the experiment
    plt.savefig(f'plots_cv_p_high_vep/phenotypes/{disease}.jpg')  # CV_P+HIGH_VEP experiment
    # plt.savefig(f'plots_cv_p/phenotypes/{disease}.jpg')    # CV_P experiment
    plt.close()